In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.spatial
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import pandas as pd
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
import torch
import pandas as pd
from pprint import pprint

import sys
import os
import glob


# Load Datasets

### Load iClinic

In [ ]:
df_iClinic =pd.read_csv('/kaggle/input/iclinic/iclinic.csv')
df_iClinic = df_iClinic.apply(lambda x: x.astype(str).str.lower())
df_iClinic = df_iClinic.rename(columns={'Question': 'question'})
df_iClinic = df_iClinic.rename(columns={'Answer': 'answer'})
df_iClinic = df_iClinic.apply(lambda x: x.astype(str).str.lower())

import pandas as pd
from sklearn.model_selection import train_test_split

df_iClinic_train, df_iClinic_test = train_test_split(df_iClinic, test_size=0.1, random_state=42)

In [ ]:
len(df_iClinic)
print(df_iClinic.columns)

### Load MedQuad

In [ ]:
PATH = '/kaggle/input/medquad/MedQuad_train.json'

In [ ]:
import json
import pandas as pd
# Open the JSON file
with open(PATH, 'r') as f:
    # Load the contents of the file into a variable
    json_data = f.read()

# Parse the JSON data into a Python dictionary
dataset_raw = json.loads(json_data)

import numpy as np
df = []
for k in dataset_raw:
  for qa in range(len(dataset_raw[k]["QAs"])):
    question =  str(dataset_raw[k]["QAs"][qa]["question"])
    answer =  str(dataset_raw[k]["QAs"][qa]["answer"])
    # if len(answer.split()) < 300:
    final = {"question": question, "answer": answer}
    df.append(final)
df = pd.DataFrame(df)
df = df.apply(lambda x: x.astype(str).str.lower())

In [ ]:
len(df)

In [ ]:
df  = df.append(df_iClinic_train, ignore_index=True)

In [ ]:
q_list = "question: " + df['question']                          # questions list to feed the model
n_list = df['answer'] + " </s>"  # answers list to feed the model

dict_data = {'source_text': q_list,
      'target_text': n_list}

df = pd.DataFrame(dict_data)
df.head()  

In [ ]:
q_list = "question: " + df_iClinic_test['question']                          # questions list to feed the model
n_list = df_iClinic_test['answer'] + " </s>"  # answers list to feed the model

dict_data = {'source_text': q_list,
      'target_text': n_list}

df_iClinic_test = pd.DataFrame(dict_data)
df_iClinic_test.head()

In [ ]:
!pip install --upgrade simplet5

* **T5 extract the answer from the question, so here we should feed the model with the question and the exact answer/s.**

In [ ]:
df.head()

In [ ]:
df['source_text'][0]

In [ ]:
df['target_text'][0]

In [ ]:
df.shape, len(df.source_text.unique()), len(df.target_text.unique())

In [ ]:
# splitting data into train and test data
train_data, val_data = train_test_split(df[:-100], test_size=0.0001)
train_data.shape, val_data.shape, df_iClinic_test.shape

In [ ]:
%%time
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-small")
model.train(train_df = train_data,
            eval_df = val_data, 
            source_max_token_len=200, 
            target_max_token_len=512, 
            batch_size=16, max_epochs=3, use_gpu=True)

In [ ]:
ll ./outputs

In [ ]:
mv "./outputs/simplet5-epoch-2-train-loss-3.0615-val-loss-2.9346" "./outputs/simplet5-iclinic+medq-epoch-3"

Stored at ./outputs/simplet5-epoch-3

In [ ]:
model.load_model("t5","./outputs/simplet5-iclinic+medq-epoch-3", use_gpu=True)# only medquad

In [ ]:
df_iClinic_test['source_text']

# Sample

In [ ]:
q_test = df_iClinic_test['source_text'][2222]
q_ans = df_iClinic_test['target_text'][2222]

print("Question: ", q_test)
print('-'*50)
print("Answer: ",q_ans)

In [ ]:
predicted_ans = model.predict(q_test)[0]

print(predicted_ans)

# Evaluation


# Evaluation data

In [ ]:
from torch.utils.data import Dataset, DataLoader
class QADatasetTest(Dataset):
    def __init__(self, data, tokenizer, max_len_out_test=512):
        self.tokenizer = tokenizer
        self.data = data
        self.max_len_out_test = max_len_out_test
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        question = self.data.loc[idx, 'source_text']
        inputs = self.tokenizer.encode_plus(
            question,
            truncation=True,
            padding='max_length' ,
            max_length=512,
            return_tensors='pt'  
        )
        return {
            'input_ids': inputs.input_ids.flatten(),
            'attention_mask': inputs.attention_mask.flatten()
        }
batch_size = 78  # Specify the desired batch size
df_iClinic_test = df_iClinic_test.reset_index()
test_dataset = QADatasetTest(df_iClinic_test, model.tokenizer, max_len_out_test=512)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4)


# num_batches = len(test_data) // batch_size

# Initialize the predicted answers list
y_pred = []


def predict(
        dl,
        max_length: int = 512,
        num_return_sequences: int = 1,
        num_beams: int = 3,
        top_k: int = 40,
        top_p: float = 0.95,
        do_sample: bool = True,
        repetition_penalty: float = 30.5,
        length_penalty: float = 1.0,
        early_stopping: bool = True,
        skip_special_tokens: bool = True,
        clean_up_tokenization_spaces: bool = True,
    ):
    device = torch.device("cuda")
    for batch in tqdm(test_dataloader, desc="Testing progress", unit="batch"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        generated_ids = model.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            num_beams=num_beams,
            max_length=max_length,
            repetition_penalty=repetition_penalty,
            length_penalty=length_penalty,
            early_stopping=early_stopping,
            top_p=top_p,
            top_k=top_k,
            num_return_sequences=num_return_sequences,
        )
#         print(generated_ids.shape, flush=True)
        preds =model.tokenizer.batch_decode(
                generated_ids,
                skip_special_tokens=skip_special_tokens,
                clean_up_tokenization_spaces=clean_up_tokenization_spaces,
            )

        y_pred.extend(preds)
    return y_pred
    
y_pred = predict(test_dataloader)

# Testing specific answers

In [ ]:
# import pickle

# with open('/kaggle/input/medquadsubset/test_queses.pickle', 'rb') as file:
#     loaded_dict = pickle.load(file)
# # shuffled_test_data = test_data.iloc[indices]
# loaded_dict
# with open('/kaggle/input/medquadsubset/test_answers.pickle', 'rb') as file:
#     loaded_ans_dict = pickle.load(file)
# # shuffled_test_data = test_data.iloc[indices]
# loaded_ans_dict
# data = {'source_text': loaded_dict, 'target_text': loaded_ans_dict}
# indices = [61,9,59,71,43]
# shuffled_test_data = pd.DataFrame(data).iloc[indices]

In [ ]:
df_iClinic_test["source_text"][71]

In [ ]:
# batch_size = 78  # Specify the desired batch size
# shuffled_test_data = shuffled_test_data.reset_index()
# test_dataset = QADatasetTest(shuffled_test_data, model.tokenizer, max_len_out_test=512)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4)
# y_pred = []
# y_pred = predict(test_dataloader)
# shuffled_test_data["source_text"]
# y_pred

In [ ]:
def save_list_to_file(data_list,questions, file_path):
    with open(file_path, 'w') as file:
        i=0
        for item in data_list:
            file.write(item + '\n')
            file.write(questions[i] + '\n')
            i=i+1
save_list_to_file(y_pred,df_iClinic_test["source_text"], './outputs/new_data_predicted_output_MedQuad+iclinic_test_iClinic.txt')

In [ ]:
y_test = df_iClinic_test["target_text"].to_list()

In [ ]:
def read_odd_lines_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        odd_lines = [line.strip() for i, line in enumerate(lines) if i % 2 != 0]
    return odd_lines
file_path = './outputs/new_data_predicted_output_MedQuad+iclinic_test_iClinic.txt'
y_pred = read_odd_lines_to_list(file_path)

In [ ]:
i=2
print("Predicted: ",y_pred[i])
print("actual : ",y_test[i])

## [BLEU SCORE](https://www.journaldev.com/46659/bleu-score-in-python)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

## [ROUGE score ](https://pypi.org/project/rouge-score/)

In [ ]:
!pip install rouge-score

# Evaluation ALL

In [ ]:
!pip install rouge-score
!pip install nltk
!pip install rouge

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import  word_tokenize
import nltk
import numpy as np
from rouge import Rouge
nltk.download('punkt')
df_iClinic_test["preds"]=y_pred

def bleu1(reference_captions, predicted_caption):
    return 100 * sentence_bleu(reference_captions, predicted_caption,
                               weights=(1, 0, 0, 0), smoothing_function=SmoothingFunction().method1)


def bleu4(reference_captions, predicted_caption):
    return 100 * sentence_bleu(reference_captions, predicted_caption,
                               weights=(0, 0, 0, 1), smoothing_function=SmoothingFunction().method1)

rouge = Rouge()

# pred_caption = word_tokenize(pred_caption.lower())

ground_truth = y_test
generated_ans = y_pred

Bleu1 = []
Bleu4 = []
rouge_1_score = []
rouge_2_score = []
rouge_L_score = []

for i in range(len(ground_truth)):
    grndAns = ground_truth[i]
    gen_ans = generated_ans[i]
    
    # BLEU SCORES
    gen_ans_ = word_tokenize(gen_ans.lower())
    grndAns_ = word_tokenize(grndAns.lower())
    bleu1s = bleu1(grndAns_, gen_ans_)
    bleu4s = bleu4(grndAns_, gen_ans_)
    Bleu1.append(bleu1s)
    Bleu4.append(bleu4s)
    
    # Rouge
    scores = rouge.get_scores(gen_ans, grndAns)
    r1s = scores[0]['rouge-1']['f'] # f1 score
    r2s = scores[0]['rouge-2']['f']
    rLs = scores[0]['rouge-l']['f']
    rouge_1_score.append(r1s)
    rouge_2_score.append(r2s)
    rouge_L_score.append(rLs)
    

print("BLEU 1 Gram: ", np.mean(Bleu1))
print("BLEU 4 Gram: ", np.mean(Bleu4))
print("ROUGE 1 Gram:", np.mean(rouge_1_score))
print("ROUGE 2 Gram:", np.mean(rouge_2_score))
print("ROUGE L Gram:", np.mean(rouge_L_score))